# Сбор данных

In [ ]:
import requests
from tqdm.auto import tqdm
from datetime import datetime
import pandas as pd
import pymysql
import mysql.connector
from mysql.connector import Error
import json
import re

In [ ]:
TOKEN = *********
VERSION = "5.130"

In [ ]:
wall_get_url = "https://api.vk.com/method/wall.get"
get_comments_url = "https://api.vk.com/method/wall.getComments"
users_get_url = "https://api.vk.com/method/users.get"

In [ ]:
group_id = {1: {'name': 'Лентач', 'id': -29534144}, 
            {2: {'name': 'Палата N6', 'id': -56106344},
            {3: {'name': 'РиаНовости', 'id': -15755094}}

Соберём функции для обкачки постов, комментариев и информации о пользователях.

In [ ]:
def get_posts(group, count, glob):    
    all_the_data = []
    offs = []
    for i in range(0+glob, count+glob, 100):
        offs.append(i)
    for off in offs:
        data = requests.get(
            wall_get_url, 
            params={
                "owner_id": group, #group - айди группы, начинается с минуса
                "count": 100,
                "v": VERSION,
                "access_token": TOKEN,
                "offset": off, 
            }
        ).json()
        if 'response' in data.keys():
            all_the_data.extend(data['response']['items'])
    return all_the_data


def get_comments(group, post_id):
    all_coms = []
    data_com = requests.get(
        get_comments_url, 
        params={
            "owner_id": group,
            "post_id": post_id,
            "v": VERSION,
            "access_token": TOKEN,
            "thread_items_count": 10
        }
    ).json()
    if 'response' in data_com.keys():
        return data_com['response']['items']
    return None


def get_user_info(needed_ids):
    user_infos = []
    for user_id in tqdm(needed_ids):
        user_info = requests.get(
            users_get_url,
            params={
                'user_ids': user_id,
                'fields': 'sex,bdate',
                'access_token': TOKEN,
                'v': VERSION
            }
        ).json()
        user_infos.append(user_info)
    return user_infos

Выкачаем посты из наших групп

In [ ]:
posts_collection = {}
all_comments_collection = {}


for public in group_id.items():
    name = public[1]['name']
    vk_id_group = public[1]['id']
    comms = []
    posts_collection[name] = get_posts(vk_id_group, 1700, 0)
    for i in tqdm(range(1, 90)):
        posts_collection[name].extend(get_posts(vk_id_group, 1700, 1700*i))  

Все посты сохраним в один json-файл.

In [ ]:
import json
with open('posts.json', 'w', encoding='utf-8') as f:
    json.dump(posts_collection, f)

Дальше посты из каждой из группы обрабатывались по отдельности

In [ ]:
for public in group_id.items():
    all_comments = {}
    name = public[1]['name']
    all_posts = posts_collection[name]
    vk_id_group = public[1]['id']
    for post in tqdm(all_posts):
        comments = get_comments(post['from_id'], post['id'])
        all_comments[post['id']] = comments 

In [ ]:
with open('ria_coms.json', 'w', encoding='utf-8') as f:
    json.dump(all_comments, f)
    
with open('palata_coms.json', 'w', encoding='utf-8') as f:
    json.dump(all_comments, f)
    
with open('lentach_coms.json', 'w', encoding='utf-8') as f:
    json.dump(all_comments, f)

In [ ]:
with open('lentach_coms.json', 'r', encoding='utf-8') as file:
    lentach_data = json.load(file)
    
with open('palata_coms.json', 'r', encoding='utf-8') as file:
    palata_data = json.load(file)
    
with open('ria_coms.json', 'r', encoding='utf-8') as file:
    ria_data = json.load(file)

Приведём json в формат списка

In [ ]:
def json_to_list(js, name):
    coms = []
    for s_coms in tqdm(js.values()):
        if s_coms:
            for com in s_coms:
                mini_dict = {}
                mini_dict['id'] = com['id']
                mini_dict['from_id'] = com['from_id']
                mini_dict['date'] = com['date']
                mini_dict['text'] = com['text']
                mini_dict['group'] = name
                coms.append(mini_dict)
                if 'thread' in com.keys():
                    if com['thread']['count'] > 0:
                        for reply in com['thread']['items']:
                            min_dict = {}
                            min_dict['id'] = reply['id']
                            min_dict['from_id'] = reply['from_id']
                            min_dict['date'] = reply['date']
                            min_dict['text'] = reply['text']
                            min_dict['group'] = name
                            coms.append(min_dict)
    return coms


lentach_coms = json_to_list(lentach_data, 'Лентач')
palata_coms = json_to_list(palata_data, 'Палата №6')
ria_coms = json_to_list(ria_data, 'РИАНовости')

In [ ]:
total_coms = []
total_coms.extend(lentach_coms)
total_coms.extend(palata_coms)
total_coms.extend(ria_coms)

df = pd.DataFrame(total_coms)
df = df.drop_duplicates()

In [ ]:
def get_date(form_data):
    return datetime.utcfromtimestamp(form_data).strftime('%d.%m.%Y %H:%M')

def get_year(d):
    return str(d).split()[0].split('.')[2]

df['datetime'] = df['date'].apply(get_date)
df['year'] = df['datetime'].apply(get_year)

In [ ]:
df['text'].replace('', np.nan, inplace=True)
df = df.dropna()
df = df.drop(['date'], axis=1)

In [ ]:
def clear_name(t):
    pattern = '\[id\d*\|.*], '
    return re.sub(pattern, '', t)

df['text'] = df['text'].apply(clear_name)

Соберём информацию о пользователях

In [ ]:
user_ids = set(list(df['from_id']))
for user in tqdm(user_ids):
    if user not in us.keys():
        user_info = requests.get(
                users_get_url,
                params={
                    'user_ids': user,
                    'fields': 'bdate,city,sex',
                    'access_token': TOKEN,
                    'v': VERSION
                }
        ).json()
        time.sleep(0.2)
        if len(user_info['response']) > 0:
            user_info = user_info['response'][0]
            us[user] = user_info

In [ ]:
bdates = []
from_ids = []
sexes = []
cities = []
names = []
for elem in us.values():
    from_id = elem['id']
    if 'bdate' in elem.keys():
        this_bdate = elem['bdate']
    else:
        this_bdate = None
    if 'sex' in elem.keys():
        this_sex = elem['sex']
    else:
        this_sex = None
    if 'city' in elem.keys():
        this_city = elem['city']['title']
    else:
        this_city = None
    if 'first_name' in elem.keys():
        this_name = elem['first_name'] + ' ' + elem['last_name']
    else:
        this_name = None
    bdates.append(this_bdate)
    sexes.append(this_sex)
    cities.append(this_city)
    names.append(this_name)
    from_ids.append(from_id)

In [ ]:
user_df = pd.DataFrame({'from_id': from_ids,
            "name": names,
            'bdate': bdates,
            'city': cities,
            'sex': sexes})

In [ ]:
mergedStuff = pd.merge(df, user_df, on=['from_id'], how='inner')

In [ ]:
final_df = mergedStuff.drop_duplicates()

In [ ]:
final_df.isna().sum()
final_df = final_df.dropna()
final_df.to_csv('final_df.csv')